In [3]:
import os
import torch
from model import FastSpeech2

In [16]:

def explore_ckpt_file(ckpt_path):
    try:
        # Load the checkpoint data
        checkpoint = torch.load(ckpt_path, map_location=torch.device('cpu'))

        # List all keys in the checkpoint dictionary (usually contains 'model_state_dict' and more)
        print("Keys in the checkpoint dictionary:\n")
        for key in checkpoint.keys():
            print(key)

        # Access the model's state_dict (modify 'model_state_dict' if different key)
        if 'model_state_dict' in checkpoint:
            state_dict = checkpoint['model_state_dict']

            # List all keys in the state_dict (these are the model's parameter names)
            print("\nKeys in the model's state_dict:\n")
            for key in state_dict.keys():
                print(key)

            # Access specific parameters (you can modify these to explore the data)
            print("\nExample: Accessing specific parameters:\n")
            parameter_name = "your_parameter_name_here"  # Change this to a specific parameter name
            if parameter_name in state_dict:
                parameter = state_dict[parameter_name]
                print(f"Parameter: {parameter_name}")
                print(f"Shape: {parameter.shape}")
                print(f"Data: {parameter}")
            else:
                print(f"Parameter {parameter_name} not found in the state_dict.")
        else:
            print("The 'model_state_dict' key not found in the checkpoint.")

    except Exception as e:
        print(f"Error: {str(e)}")

# if __name__ == "__main__":
    # Replace 'path/to/your/checkpoint.ckpt' with the actual path to your .ckpt file
ckpt_file_path = './NovaEmo.pth'
#explore_ckpt_file(ckpt_file_path)


In [17]:
checkpoint = torch.load(ckpt_file_path, map_location=torch.device('cpu'))

In [18]:
import yaml
model_config = yaml.load(open('./config/Nova/model.yaml',
                              "r"), Loader=yaml.FullLoader)
preprocess_config = yaml.load(open('./config/Nova/preprocess.yaml',
                              "r"), Loader=yaml.FullLoader)

In [6]:
# model_config

In [7]:
# preprocess_config

In [8]:
# fp = FastSpeech2(model_config=model_config, preprocess_config=preprocess_config)
# #ckpt_file_path = '/workspace/nemo/vol/FastSpeech2/output/ckpt/RAVDESS/800000.pth.tar'
# # checkpoint = torch.load(ckpt_file_path, map_location=torch.device('cuda'))
# checkpoint = torch.load(ckpt_file_path, map_location=torch.device('cpu'))
# fp.load_state_dict(checkpoint['model'])
# fp = fp.to('cuda')

In [8]:
old_speakers = checkpoint['model']['speaker_emb.weight']

In [9]:
old_speakers.size()

torch.Size([2324, 512])

In [10]:
emotion_tensor = torch.randn(13, 512)
speaker_tensor = torch.randn(210, 512)

In [10]:
old_emotions = checkpoint['model']['emotion_emb.weight']

In [11]:
old_emotions.size()

torch.Size([25, 512])

In [15]:
new_speakers = speaker_tensor
new_emotions = torch.cat((old_emotions, emotion_tensor), dim=0)

In [31]:
new_speakers.size()

torch.Size([1, 512])

In [32]:
new_emotions.size()

torch.Size([26, 512])

In [12]:
checkpoint['model']['speaker_emb.weight'] =  speaker_tensor

In [11]:
checkpoint['model']['emotion_emb.weight'] = emotion_tensor

In [19]:
fp = FastSpeech2(model_config=model_config, preprocess_config=preprocess_config)

In [20]:
fp.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [14]:
torch.save(checkpoint, 'NovaEmo.pth')

In [40]:
for name, param in fp.named_parameters():
    if 'speaker' in name:
        print(name, param.requires_grad)
        continue
    param.requires_grad = False

speaker_emb.weight True


In [21]:
for name, param in fp.named_parameters():
    print(name, param.requires_grad)

encoder.position_enc False
encoder.src_word_emb.weight True
encoder.layer_stack.0.slf_attn.w_qs.weight True
encoder.layer_stack.0.slf_attn.w_qs.bias True
encoder.layer_stack.0.slf_attn.w_ks.weight True
encoder.layer_stack.0.slf_attn.w_ks.bias True
encoder.layer_stack.0.slf_attn.w_vs.weight True
encoder.layer_stack.0.slf_attn.w_vs.bias True
encoder.layer_stack.0.slf_attn.layer_norm.weight True
encoder.layer_stack.0.slf_attn.layer_norm.bias True
encoder.layer_stack.0.slf_attn.fc.weight True
encoder.layer_stack.0.slf_attn.fc.bias True
encoder.layer_stack.0.pos_ffn.w_1.weight True
encoder.layer_stack.0.pos_ffn.w_1.bias True
encoder.layer_stack.0.pos_ffn.w_2.weight True
encoder.layer_stack.0.pos_ffn.w_2.bias True
encoder.layer_stack.0.pos_ffn.layer_norm.weight True
encoder.layer_stack.0.pos_ffn.layer_norm.bias True
encoder.layer_stack.1.slf_attn.w_qs.weight True
encoder.layer_stack.1.slf_attn.w_qs.bias True
encoder.layer_stack.1.slf_attn.w_ks.weight True
encoder.layer_stack.1.slf_attn.w_ks.

In [44]:
256 ** -0.5

0.0625